# 深層学習を用いた言語モデル

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
ニューラルネットワークを用いて、ある単語から次の単語を予測するモデルを作成し、深層学習を活用した言語モデル実装の基礎を学ぶ。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab, build_vocab_from_iterator
from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## データセット

wiki40b

In [2]:
with tf.device('/cpu:0'):
    ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

In [3]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [4]:
data = data[:100]


---

## 前処理

テキストをNNで扱える形に変換する。

### トークン化

文章をトークンごとに分割する。トークンとは文章を構成する最小の単位で、単語や句読点などが該当する。  
トークン化は分かち書きと似た意味であるが、言語モデルの領域で最小単位をトークンと呼ぶことが一般的なことや、単純にトークン化と呼ぶことが多いことから、トークン化と呼ぶ。

トークン化には、分かち書き同様形態素解析を用いる。各文章をトークンのリストとして格納する。

In [5]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
for sentence in data:
    tokens = tagger.parse(sentence).strip().split(' ') # トークンのリスト
    data_tokens.append(tokens)

data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### ID化

NNでは文字列を扱えないので、トークン1つ1つにIDを割り当てる。

<br>

torchtextの`vocab`モジュールを使用する。

[torchtext.vocab — Torchtext 0.15.0 documentation](https://pytorch.org/text/stable/vocab.html#)

In [6]:
from torchtext.vocab import vocab, build_vocab_from_iterator

単語とその出現頻度を表した辞書を入力し、`Vocab`オブジェクトを作成する。`min_freq`で指定した出現頻度以下の単語は無視される。

In [7]:
v = vocab({'私': 4, 'りんご': 1, '食べる': 2, '好き': 3}, min_freq=1)
v(['私', 'りんご', '好き'])

[0, 1, 3]

`build_vocab_from_iterator()`でトークンのリスト（のリスト）から`vocab`オブジェクトを作成できる。

In [8]:
v = build_vocab_from_iterator([
    ['私', 'は', 'りんご', 'が', '好き'],
    ['私', 'は', 'バナナ', 'を', '食べる'],
    ['今日', 'は', '晴れ', 'です']
])
v.get_itos() # vocabrary

['は', '私', 'が', 'です', 'りんご', 'を', 'バナナ', '今日', '好き', '晴れ', '食べる']

これを使ってトークン列をID列に変換する。

In [9]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

n_vocab = len(v)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 4602


[17, 3681, 3742, 2, 4, 3873, 4415, 47, 19, 70]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        x, y = [], []
        for ids in data_ids:
            for id1, id2 in zip(ids[:-1], ids[1:]):
                x.append(id1)
                y.append(id2)
        self.x = x
        self.y = y
        self.n_data = len(x)

    def __len__(self):
        return self.n_data

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 21128
tensor([ 402,   44,   26,    6,  650,  391,  402,   43, 2383,   16,   74,  617,
          26, 3054, 4162,   11,    2,  234,   56,  393,  575,  645,    6,    1,
        3724, 4359,    0,    6,  196, 2961,  618,  619,   17, 3039,    0,    4,
          30, 3430,  255,  306,    5,   12,  336,    1,    1,   46,    0,   19,
           5,    7,    9,    2,    1,   55,   22,   63,    6,    3, 1106,   19,
           4,    4, 1793,  119])
tensor([  62,   52,  529,    3, 2482,  174,  277,  150,    0,   74,  676,  119,
           0,   21,    1,    1,  133,    0,   28,    1, 3117,   14, 4234, 4446,
           2,   37, 2850,    0,    9,   21,    5,  115,   94,    6, 2685,   94,
          60, 4071,    1, 3361, 4174,    2,   39,  976,  190,   44,  365,    4,
        3817,  450,  933,   24, 2932,    7,  697,   32,    3,  136,   19,    7,
         733,  236,  168,    7])



---

## モデル構築

ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDはカテゴリ変数なので、入力時はone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルとする。つまりこのタスクは単語の分類問題となる。

In [11]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, hidden_dim):
        super().__init__()
        self.n_vocab = n_vocab
        self.net = nn.Sequential(
            nn.Linear(n_vocab, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_vocab)
        )

    def forward(self, x):
        x = F.one_hot(x, self.n_vocab).to(torch.float32)
        y = self.net(x)
        return y


---

## 学習

通常の分類モデルと同じように学習する。

In [12]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, dataloader, n_epochs):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_epoch = 0
        for x, t in tqdm(dataloader, desc=f'{epoch}/{n_epochs}', leave=False):
            optimizer.zero_grad()
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()
        loss = loss_epoch/len(dataloader)
        print(f'{epoch}/{n_epochs} loss: {loss}', flush=True)

In [13]:
hidden_dim = 512
model = LanguageModel(n_vocab, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
train(model, optimizer, dataloader, 20)

1/20 loss: 6.617097869020214


2/20 loss: 5.616453633207569


3/20 loss: 5.075052810939777


4/20 loss: 4.563654917604613


5/20 loss: 4.121555262101741


...

16/20 loss: 2.7184031924452308


17/20 loss: 2.708634745318364


18/20 loss: 2.692874877474459


19/20 loss: 2.686548812511824


20/20 loss: 2.683840165325522



---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。ある単語からの次の単語の予測を繰り返すことで文章を生成する。初めの単語だけはこちらで指定する。  
以下の条件を満たすまで単語の生成を続ける。
- 単語数が指定した限界に達する
- 読点が出力される

In [15]:
end_token_id = v.get_stoi()['。'] # 読点のID

def generate_sentence(model, start_word, max_len=30):
    model.eval()
    token_id = v.get_stoi()[start_word]
    token_ids = [token_id]

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != end_token_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor(token_id).unsqueeze(0).to(device)

        # 次の単語の確率分布を予測
        y = model(x)[0]
        y = F.softmax(y, dim=0)

        # 分布に基づいたサンプリングを行う
        token_id = random.choices(range(len(y)), weights=y)[0]
        token_ids.append(token_id)

    tokens = [v.get_itos()[i] for i in token_ids]
    sentence = ''.join(tokens)
    return sentence

In [16]:
start_words = ['今日', '地球', '科学', '人']
for start_word in start_words:
    print(generate_sentence(model, start_word))

今日の美術統括責任者が、そこからと楽しい関係して直接の区別され、日本に直接目標としている。
地球形が新設した。
科学奴隷」から絵の情報提供させている。
人ほどの小説集と2007年に腎不全の場合を開き、YouTubeなどが争うは実家で丸儲け）で並んだからは、



---

## 埋め込み層

*Embedding Layer*

NN内部における、単語IDに対する「one-hotベクトル化→線形変換」を行う部分をまとめて**埋め込み層**と表す。  
そもそも、これらの処理は単語IDを受け取って対応するベクトルを出力すること=**単語のベクトル化**と同義である。自然言語処理の世界では単語のベクトル化を**単語の埋め込み**と表現する。

線形変換を行う全結合層は各単語の埋め込み表現（単語ベクトル）を所持していることになる。それらは重みから確認できる。

In [17]:
class Embedding(nn.Module):
    def __init__(self, n_vocab, embed_dim):
        super().__init__()
        self.n_vocab = n_vocab
        self.fc = nn.Linear(n_vocab, embed_dim, bias=False)

    def forward(self, x):
        x = F.one_hot(x, self.n_vocab).to(torch.float32)
        h = self.fc(x)
        return h

embedding = Embedding(n_vocab=3, embed_dim=5)

x = torch.tensor([0, 1, 2])
h = embedding(x)

print(h, '\n') # 埋め込み層からの出力
print(embedding.fc.weight.T) # 全結合層の重み

tensor([[ 0.2873, -0.2740,  0.1545, -0.5471, -0.3373],
        [-0.4933, -0.2826, -0.4215,  0.1007,  0.4101],
        [-0.3496,  0.4926,  0.2063,  0.4489, -0.5141]], grad_fn=<MmBackward0>) 

tensor([[ 0.2873, -0.2740,  0.1545, -0.5471, -0.3373],
        [-0.4933, -0.2826, -0.4215,  0.1007,  0.4101],
        [-0.3496,  0.4926,  0.2063,  0.4489, -0.5141]],
       grad_fn=<PermuteBackward0>)


`PyTorch`には`torch.nn.Embedding`というクラスが実装されているので、それを使うと良い。  
[Embedding — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [18]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Embedding(n_vocab, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_vocab)
        )

    def forward(self, x):
        y = self.net(x)
        return y

実際に、言語モデルの学習によって得られる単語ベクトルは良い埋め込み表現として機能することが知られている。単語の埋め込み表現を得ることを目的として言語モデルを学習させることもある。word2vecなどが該当し、これについてはおまけの章で取り上げる。


---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### BOS

*Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### EOS

*End of Sentence*

文章の終わりを意味するトークン。  
先程の例では読点が出た場合に生成を止めたが、本来読点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けることが多い。

<br>

では特殊トークンを使って学習させてみよう。使う特殊トークン上の二つ。

In [19]:
data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

全ての文章にBOSとEOSを追加する。  
他のトークンと重複しないような文字列を指定する。

In [20]:
BOS, EOS = '<BOS>', '<EOS>'
for tokens in data_tokens:
    tokens.insert(0, BOS)
    tokens.append(EOS)

data_tokens[0][:10] # example

['<BOS>', '「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」']

残りの学習は同じ

In [21]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [22]:
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [23]:
train(model, optimizer, dataloader, 20)

1/20 loss: 6.270350737486057


2/20 loss: 4.045359416636164


3/20 loss: 3.567927011472736


4/20 loss: 3.3716143755141847


5/20 loss: 3.2500009079893193


...

16/20 loss: 2.8596271961748956


17/20 loss: 2.8511292149206837


18/20 loss: 2.8468612114826364


19/20 loss: 2.8403869474719383


20/20 loss: 2.8332042429975406


では文章を生成する。初めのトークンはBOSとし、EOSが出力されるまで生成を続ける。

In [24]:
end_token_id = v.get_stoi()[EOS]

def generate_sentence(model, max_len=50):
    model.eval()
    token_id = v.get_stoi()[BOS]
    token_ids = []

    for _ in range(max_len):
        x = torch.tensor(token_id).unsqueeze(0).to(device)
        y = model(x)[0]
        y = F.softmax(y, dim=0)
        token_id = random.choices(range(len(y)), weights=y)[0]
        if token_id == end_token_id:
            break
        token_ids.append(token_id)

    tokens = [v.get_itos()[i] for i in token_ids]
    sentence = ''.join(tokens)
    return sentence

In [25]:
for _ in range(5):
    print(generate_sentence(model))

ゲーム内に変更されて週にワープされ、榎本は富士山頂に留学のが1692年半独立の教員の怒りに王国の長女とオイローパによる浅間神社のモテットを訪れた。593年大会」の
養成所ニ執心シテ、、大舘は同時についてジャイアント馬場を取ればなかった。元禄年生放送終了後を消費してはメダルーサ級殲滅型天体にそれはケンブリッジの洗礼を中心とは、系統
本作の放送局である少年聖歌隊員において待ても実施さから地球連合艦隊司令官の販売を散らしていた。子供時代にシュートで提供していった歌詞は、ピクチャーレーベルであり、
ファンタシースター感謝祭2016年）に移されない。2017年に大きな無関係を受けている。その』シリーズ、それまでこの作品《詩編のDNAを出演した。また、舞台とし、車路墘
俳協演劇研究所ニ地ヲシメ…」5世（1692年（1軍がいる。父は歌劇《スターシリーズに増加（ともにドミニカ共和政時代の以降凍結した曲以上の高い「ぽすれん」とし
